# Olat examples
Q1410031 (Tom Meets Zizou) - The publication date of Tom Meets Zizou is 2011-01-01. 
[Crowd, inter-rater agreement 0.040, The answer distribution for this specific task was 3 reject votes, 0 support votes] 
- in dataset 5 times
these 2 workers must go away: LPQMUDT6729 (approval rate is 40%) and ZZHL098SA43 (20 mal for 10s)\

Q171300 (The Princess and the Frog)
The box office of The Princess and the Frog is 267000000. 
[Crowd, inter-rater agreement 0.236, The answer distribution for this specific task was 2 support votes, 1 reject vote] 
- in dataset 5 times
- 2 reject votes (2134U7HKDMM and 1726JMZQW) - one must away (most likely 2134U7HKDMM - worktime 2s 21 times)
- 3 support votes (one must away): 2133ICYWE97, 928UJANWZ12, 2133U7HKDLO (this has only 40% approval rate)


Q223596 X-Men: First Class? 
The executive producer is Sheryl Lee Ralph. 
[Crowd, inter-rater agreement 0.199, The answer distribution for this specific task was 2 support votes, 1 reject votes] 
- in dataset 5 times
- 2 reject votes: HHCKW1111 and WWHL098SA43 (WWHL098SA43 - 2s 10 times)
- 3 support votes: AALKMII97, GGUI83657S and QZAHIFT8263 (only 40% approval on another tasks)

In [1]:
import pandas as pd

# Load the TSV file into a DataFrame
path = './../too_large_dataset/crowd_data/crowd_data_olat_P344FullstopCorrected.tsv'
data = pd.read_csv(path, sep='\t')

# Display the first few rows to understand the structure and content of the file
# print the size
print(data.shape)
data.head()

(305, 16)


,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
0,1,7QT,Is this triple correct or incorrect?,$0.50,1,2133ICYWE97,Submitted,60,99%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
1,1,7QT,Is this triple correct or incorrect?,$0.50,2,2133U7HKDLO,Submitted,40,40%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,yes,yes
2,1,7QT,Is this triple correct or incorrect?,$0.50,3,928UJANWZ12,Submitted,50,98%,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
3,1,7QT,Is this triple correct or incorrect?,$0.50,4,1726JMZQW,Submitted,80,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
4,1,7QT,Is this triple correct or incorrect?,$0.50,5,2134U7HKDMM,Submitted,2,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN


# Cleaning

In [2]:
# remove all where the Lifetime approval rate is > 50%
# you need to convert 50% in the value into a float or int
without_good_approval_rate = data.copy()
without_good_approval_rate['LifetimeApprovalRate'] = without_good_approval_rate['LifetimeApprovalRate'].str.rstrip('%').astype('float')
without_good_approval_rate = without_good_approval_rate[without_good_approval_rate['LifetimeApprovalRate'] >= 50]
print(without_good_approval_rate.shape)

(244, 16)


In [3]:
# remove all of that return consistent data
consistent_workers = without_good_approval_rate.groupby('WorkerId')['AnswerLabel'].nunique()
malicious_workers = consistent_workers[consistent_workers == 1].index
print(malicious_workers)
filtered_data = without_good_approval_rate[~without_good_approval_rate['WorkerId'].isin(malicious_workers)]
print(filtered_data.shape)
filtered_data.head(5)

Index([], dtype='object', name='WorkerId')
(244, 16)


,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
0,1,7QT,Is this triple correct or incorrect?,$0.50,1,2133ICYWE97,Submitted,60,99.0,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
2,1,7QT,Is this triple correct or incorrect?,$0.50,3,928UJANWZ12,Submitted,50,98.0,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
3,1,7QT,Is this triple correct or incorrect?,$0.50,4,1726JMZQW,Submitted,80,70.0,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
4,1,7QT,Is this triple correct or incorrect?,$0.50,5,2134U7HKDMM,Submitted,2,70.0,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
5,2,7QT,Is this triple correct or incorrect?,$0.50,6,2134U7HKDMM,Submitted,2,70.0,wd:Q603545,wdt:P2142,4300000,2.0,INCORRECT,NaN,NaN


In [4]:
import plotly.express as px
import pandas as pd

# Count the occurrences of each (WorkerId, WorkTimeInSeconds) pair
aggregated_data = filtered_data.groupby(['WorkerId', 'WorkTimeInSeconds']).size().reset_index(name='Count')

# Create a scatter plot with bubble size based on the count
fig = px.scatter(aggregated_data, 
                 x='WorkerId', 
                 y='WorkTimeInSeconds', 
                 size='Count',
                 title="Work Time Distribution per Worker (Bubble Size Reflects Count)",
                 labels={'WorkerId': 'Worker ID', 'WorkTimeInSeconds': 'Work Time (Seconds)', 'Count': 'Number of Points'},
                 template='plotly_white')

fig.update_xaxes(tickangle=90)
fig.show()


In [5]:
# # remove all which have no variance in the WorkTimeInSeconds
# consistent_workers = filtered_data.groupby('WorkerId')['WorkTimeInSeconds'].nunique()

# consistent_workers = consistent_workers[consistent_workers == 1].index
# # print count of consistent workers
# print(len(consistent_workers))
# without_cw = filtered_data[~filtered_data['WorkerId'].isin(consistent_workers)]
# print(without_cw.shape)
# without_cw.head(3)

In [ ]:
# import plotly.express as px
# import pandas as pd

# # Count the occurrences of each (WorkerId, WorkTimeInSeconds) pair
# aggregated_data = without_cw.groupby(['WorkerId', 'WorkTimeInSeconds']).size().reset_index(name='Count')

# # Create a scatter plot with bubble size based on the count
# fig = px.scatter(aggregated_data, a
#                  x='WorkerId', 
#                  y='WorkTimeInSeconds', 
#                  size='Count',
#                  title="Work Time Distribution per Worker (Bubble Size Reflects Count)",
#                  labels={'WorkerId': 'Worker ID', 'WorkTimeInSeconds': 'Work Time (Seconds)', 'Count': 'Number of Points'},
#                  template='plotly_white')

# fig.update_xaxes(tickangle=90)
# fig.show()


NameError: name 'without_cw' is not defined

In [8]:
# remove all from filtered data where the seconds is > 10s
more_than10s = filtered_data[filtered_data['WorkTimeInSeconds'] > 10]
print(more_than10s.shape)

(183, 16)


# Kappa and similar

In [9]:
# %pip install statsmodels

In [10]:
from statsmodels.stats.inter_rater import fleiss_kappa

df_to_use = more_than10s

summary = df_to_use.groupby(['HITId', 'AnswerLabel']).size().unstack(fill_value=0)

# Include the RDF triple components
triplet_info = df_to_use.groupby('HITId').agg({
    'Input1ID': 'first',
    'Input2ID': 'first',
    'Input3ID': 'first'
})

# Merge triplet information into the summary
summary = summary.merge(triplet_info, left_index=True, right_index=True)

# Add Fleiss' Kappa per HIT batch (HITTypeId)
fleiss_data = df_to_use.pivot_table(index='HITId', columns='AnswerLabel', aggfunc='size', fill_value=0)
fleiss_kappas = df_to_use.groupby('HITTypeId').apply(
    lambda group: fleiss_kappa(fleiss_data.loc[group['HITId'].unique()].values, method='fleiss')
)
print(fleiss_kappas)

# Map Fleiss' Kappa to each HITId's batch
hit_to_batch = df_to_use.set_index('HITId')['HITTypeId'].to_dict()
summary['Fleiss Kappa'] = summary.index.map(lambda hit_id: fleiss_kappas.get(hit_to_batch.get(hit_id), None))

# Reset index for a clean table
summary = summary.reset_index().rename(columns={'CORRECT': 'nr_correct', 'INCORRECT': 'nr_incorrect'})

# Display the final summary table
summary

HITTypeId
7QT    0.236364
8QT    0.040000
9QT    0.199110
dtype: float64


C:\Users\adam\AppData\Local\Temp\ipykernel_3036\28423049.py:19: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,HITId,nr_correct,nr_incorrect,Input1ID,Input2ID,Input3ID,Fleiss Kappa
0,1,2,1,wd:Q11621,wdt:P2142,792910554,0.236364
1,2,3,0,wd:Q603545,wdt:P2142,4300000,0.236364
2,3,2,1,wd:Q16911843,wdt:P577,2014-01-18,0.236364
3,4,0,3,wd:Q132863,wdt:P2142,969023261,0.236364
4,5,3,0,wd:Q1628022,wdt:P577,1951-01-01,0.236364
...,...,...,...,...,...,...,...
56,57,2,1,wd:Q223596,wdt:P1431,wd:Q457180,0.199110
57,58,0,3,wd:Q943992,wdt:P161,wd:Q160432,0.199110
58,59,2,1,wd:Q1893555,wdt:P272,wd:Q48784114,0.199110
59,60,2,1,wd:Q21060270,wdt:P27,wd:Q916,0.199110


In [11]:
# export the data to processed_crows.csv
summary.to_csv('processed_crowd.csv', index=False)